# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 18 2022 10:06AM,237530,20,SSNA-2173538714,Sartorius Stedim North America,Released
1,May 18 2022 10:06AM,237529,10,DC-0499-05172022,"Herbivore Botanicals, LLC.",Released
2,May 18 2022 10:06AM,237529,10,DC-0699-05172022,"Herbivore Botanicals, LLC.",Released
3,May 18 2022 9:59AM,237525,10,0085676077,ISDIN Corporation,Released
4,May 18 2022 9:59AM,237525,10,0085676076,ISDIN Corporation,Released
5,May 18 2022 9:59AM,237525,10,0085676074,ISDIN Corporation,Released
6,May 18 2022 9:59AM,237525,10,0085676075,ISDIN Corporation,Released
7,May 18 2022 9:59AM,237525,10,0085676144,ISDIN Corporation,Released
8,May 18 2022 9:59AM,237525,10,0085676163,ISDIN Corporation,Released
9,May 18 2022 9:59AM,237525,10,0085676165,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
86,237526,Released,1
87,237527,Released,1
88,237528,Released,1
89,237529,Released,2
90,237530,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
237526,NaN,NaN,1.0
237527,NaN,NaN,1.0
237528,NaN,NaN,1.0
237529,NaN,NaN,2.0
237530,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237385,0.0,0.0,1.0
237388,0.0,0.0,1.0
237394,0.0,0.0,1.0
237399,0.0,0.0,28.0
237404,0.0,6.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237385,0,0,1
237388,0,0,1
237394,0,0,1
237399,0,0,28
237404,0,6,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237385,0,0,1
1,237388,0,0,1
2,237394,0,0,1
3,237399,0,0,28
4,237404,0,6,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237385,,,1
1,237388,,,1
2,237394,,,1
3,237399,,,28
4,237404,,6,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 18 2022 10:06AM,237530,20,Sartorius Stedim North America
1,May 18 2022 10:06AM,237529,10,"Herbivore Botanicals, LLC."
3,May 18 2022 9:59AM,237525,10,ISDIN Corporation
44,May 18 2022 9:55AM,237528,10,Beach Patient Assistance
45,May 18 2022 9:54AM,237521,10,ISDIN Corporation
81,May 18 2022 9:53AM,237527,10,"Nextsource Biotechnology, LLC"
82,May 18 2022 9:53AM,237526,10,ISDIN Corporation
83,May 18 2022 9:41AM,237524,15,"Virtus Pharmaceuticals, LLC"
84,May 18 2022 9:41AM,237523,15,"Person & Covey, Inc."
92,May 18 2022 9:37AM,237520,15,"Person & Covey, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 18 2022 10:06AM,237530,20,Sartorius Stedim North America,,,1
1,May 18 2022 10:06AM,237529,10,"Herbivore Botanicals, LLC.",,,2
2,May 18 2022 9:59AM,237525,10,ISDIN Corporation,,,41
3,May 18 2022 9:55AM,237528,10,Beach Patient Assistance,,,1
4,May 18 2022 9:54AM,237521,10,ISDIN Corporation,,,36
5,May 18 2022 9:53AM,237527,10,"Nextsource Biotechnology, LLC",,,1
6,May 18 2022 9:53AM,237526,10,ISDIN Corporation,,,1
7,May 18 2022 9:41AM,237524,15,"Virtus Pharmaceuticals, LLC",,,1
8,May 18 2022 9:41AM,237523,15,"Person & Covey, Inc.",,,8
9,May 18 2022 9:37AM,237520,15,"Person & Covey, Inc.",,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 10:06AM,237530,20,Sartorius Stedim North America,1,,
1,May 18 2022 10:06AM,237529,10,"Herbivore Botanicals, LLC.",2,,
2,May 18 2022 9:59AM,237525,10,ISDIN Corporation,41,,
3,May 18 2022 9:55AM,237528,10,Beach Patient Assistance,1,,
4,May 18 2022 9:54AM,237521,10,ISDIN Corporation,36,,
5,May 18 2022 9:53AM,237527,10,"Nextsource Biotechnology, LLC",1,,
6,May 18 2022 9:53AM,237526,10,ISDIN Corporation,1,,
7,May 18 2022 9:41AM,237524,15,"Virtus Pharmaceuticals, LLC",1,,
8,May 18 2022 9:41AM,237523,15,"Person & Covey, Inc.",8,,
9,May 18 2022 9:37AM,237520,15,"Person & Covey, Inc.",11,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 10:06AM,237530,20,Sartorius Stedim North America,1,,
1,May 18 2022 10:06AM,237529,10,"Herbivore Botanicals, LLC.",2,,
2,May 18 2022 9:59AM,237525,10,ISDIN Corporation,41,,
3,May 18 2022 9:55AM,237528,10,Beach Patient Assistance,1,,
4,May 18 2022 9:54AM,237521,10,ISDIN Corporation,36,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 10:06AM,237530,20,Sartorius Stedim North America,1.0,NaN,NaN
1,May 18 2022 10:06AM,237529,10,"Herbivore Botanicals, LLC.",2.0,NaN,NaN
2,May 18 2022 9:59AM,237525,10,ISDIN Corporation,41.0,NaN,NaN
3,May 18 2022 9:55AM,237528,10,Beach Patient Assistance,1.0,NaN,NaN
4,May 18 2022 9:54AM,237521,10,ISDIN Corporation,36.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 18 2022 10:06AM,237530,20,Sartorius Stedim North America,1.0,0.0,0.0
1,May 18 2022 10:06AM,237529,10,"Herbivore Botanicals, LLC.",2.0,0.0,0.0
2,May 18 2022 9:59AM,237525,10,ISDIN Corporation,41.0,0.0,0.0
3,May 18 2022 9:55AM,237528,10,Beach Patient Assistance,1.0,0.0,0.0
4,May 18 2022 9:54AM,237521,10,ISDIN Corporation,36.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4749632,169.0,28.0,33.0
15,2850051,73.0,61.0,0.0
16,712247,2.0,1.0,0.0
18,5461616,38.0,0.0,0.0
19,949803,19.0,14.0,0.0
20,2612089,61.0,11.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4749632,169.0,28.0,33.0
1,15,2850051,73.0,61.0,0.0
2,16,712247,2.0,1.0,0.0
3,18,5461616,38.0,0.0,0.0
4,19,949803,19.0,14.0,0.0
5,20,2612089,61.0,11.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,169.0,28.0,33.0
1,15,73.0,61.0,0.0
2,16,2.0,1.0,0.0
3,18,38.0,0.0,0.0
4,19,19.0,14.0,0.0
5,20,61.0,11.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,169.0
1,15,Released,73.0
2,16,Released,2.0
3,18,Released,38.0
4,19,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20
Status,,,,,,
Completed,33.0,0.0,0.0,0.0,0.0,0.0
Executing,28.0,61.0,1.0,0.0,14.0,11.0
Released,169.0,73.0,2.0,38.0,19.0,61.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20
0,Completed,33.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,61.0,1.0,0.0,14.0,11.0
2,Released,169.0,73.0,2.0,38.0,19.0,61.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20
0,Completed,33.0,0.0,0.0,0.0,0.0,0.0
1,Executing,28.0,61.0,1.0,0.0,14.0,11.0
2,Released,169.0,73.0,2.0,38.0,19.0,61.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()